# pd.DataFrame

`pd.DataFrame` --- по сути дела таблица, на которую можно смотреть как на объединение столбцов `pd.Series` с выравниванием по общему индексу. 


Можно изменять содержимое ячеек таблицы, добавление новых строк --- дорогостоящая операция (пересоздание таблицы), но добавление столбцов осуществляется на месте.

## Создание таблицы

Есть множество способов создать датафрейм из уже существующих объектов python. 

Один самых удобных --- используя словари. 

Если ключи в словаре соответствуют именам столбцов, а значения --- списки, `NumPy` массивы, `pandas` серии данных этих столбцов, то используется конструктор `pd.DataFrame`. При этом если в значениях присутствуют объекты `pd.Series`, то индекс созданного датафрейма будет объединением индексов серий, а данные всех столбцов выравниваются по меткам этого индекса.

In [3]:
import pandas as pd
import numpy as np

col1 = pd.Series([1, 2], index=["a", "b"])
col2 = pd.Series([3, 4], index=["b", "a"])
col3 = pd.Series([5, 6, 7], index=["a", "b", "c"])

d = {
    'col1': col1, 
    'col2': col2,
    'col3': col3,
}

df = pd.DataFrame(d)
df

,col1,col2,col3
a,1.0,4.0,5
b,2.0,3.0,6
c,NaN,NaN,7


Если значения в словаре списки или массивы, то они должны быть одной длинны.  В созданной таблице данные в столбцах идут в том же порядке, что и в исходных списках/массивах (без выравнивания), что и в списках. Индекс созданной таблицы по умолчанию --- `RangeIndex`, но можно явно указать его при создании опциональным параметром `index`.

In [4]:
col1 = np.array([1, 2])
col2 = [3, 4]

d = {
    "col1": col1,
    "col2": col2
}

df = pd.DataFrame(d, index=["a", "b"])
df

,col1,col2
a,1,3
b,2,4


[pd.DataFrame.form_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html#pandas.DataFrame.from_dict) --- более специализированный метод для создания датафрейма из словаря. В предыдущих двух примерах он сработал бы аналогично, но его имя длиннее. Его преимущество заключается, например, в том, что он позволяет создать датафрейм из словаря, в котором ключи --- метки строк (индекс), а значения --- строки будущей таблицы.

Для этого необходимо указать в качестве параметра `orient` строку `index`. Далее все аналогично. Если строки представлены в виде `pd.Series`, то метки попадут в названия столбцов с выравниванием данных. Если это простые массивы, то имена столбцов --- `RangeIndex`, но можно явно указать имена столбцов по параметру `columns`.

In [5]:
row1 = pd.Series([1, 2], index=["a", "b"])
row2 = pd.Series([3, 4], index=["b", "a"])

d = {
    "row1": row1,
    "row2": row2,
}

pd.DataFrame.from_dict(d, orient="index")

,a,b
row1,1,2
row2,4,3


Если данные представлены в виде какой-то двухмерной структуры, в которой отсутствуют метки и строк и столбцов, то можно воспользоваться методом `pd.DataFrame`, при этом индекс и имена столбцов можно указать соответствующими параметрами `index` и `columns`.  

In [6]:
data = [
    [1, 2],
    [3, 4],
    [5, 6]
]

pd.DataFrame(data, index=["row1", "row2", "row3"], columns=["col1", "col2"])

,col1,col2
row1,1,2
row2,3,4
row3,5,6


## Хранение и чтение таблиц 

Для хранения таблиц чаще всего используются 

Гораздо чаще датафрейм получается, как результат чтение таблицы из файла. Чаще всего такие таблицы хранятся в формате [csv](https://ru.wikipedia.org/wiki/CSV) или в форматах таблиц `excel`.


> Строка таблицы соответствует строке текста, которая содержит одно или несколько полей, разделенных запятыми.


```
brand,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.9,2.62,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.9,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.32,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.46,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.57,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.19,20.0,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.15,22.9,1,0,4,2
Merc 280,19.2,6,167.6,123,3.92,3.44,18.3,1,0,4,4
Merc 280C,17.8,6,167.6,123,3.92,3.44,18.9,1,0,4,4
Merc 450SE,16.4,8,275.8,180,3.07,4.07,17.4,0,0,3,3
Merc 450SL,17.3,8,275.8,180,3.07,3.73,17.6,0,0,3,3
Merc 450SLC,15.2,8,275.8,180,3.07,3.78,18.0,0,0,3,3
Cadillac Fleetwood,10.4,8,472.0,205,2.93,5.25,17.98,0,0,3,4
Lincoln Continental,10.4,8,460.0,215,3.0,5.424,17.82,0,0,3,4
Chrysler Imperial,14.7,8,440.0,230,3.23,5.345,17.42,0,0,3,4
Fiat 128,32.4,4,78.7,66,4.08,2.2,19.47,1,1,4,1
Honda Civic,30.4,4,75.7,52,4.93,1.615,18.52,1,1,4,2
Toyota Corolla,33.9,4,71.1,65,4.22,1.835,19.9,1,1,4,1
Toyota Corona,21.5,4,120.1,97,3.7,2.465,20.01,1,0,3,1
Dodge Challenger,15.5,8,318.0,150,2.76,3.52,16.87,0,0,3,2
AMC Javelin,15.2,8,304.0,150,3.15,3.435,17.3,0,0,3,2
Camaro Z28,13.3,8,350.0,245,3.73,3.84,15.41,0,0,3,4
Pontiac Firebird,19.2,8,400.0,175,3.08,3.845,17.05,0,0,3,2
Fiat X1-9,27.3,4,79.0,66,4.08,1.935,18.9,1,1,4,1
Porsche 914-2,26.0,4,120.3,91,4.43,2.14,16.7,0,1,5,2
Lotus Europa,30.4,4,95.1,113,3.77,1.513,16.9,1,1,5,2
Ford Pantera L,15.8,8,351.0,264,4.22,3.17,14.5,0,1,5,4
Ferrari Dino,19.7,6,145.0,175,3.62,2.77,15.5,0,1,5,6
Maserati Bora,15.0,8,301.0,335,3.54,3.57,14.6,0,1,5,8
Volvo 142E,,,,,,,,,,,

```

Пример простейшего `csv` файла приведен выше. В этом примере первая строка выступает в качестве заголовка, т.е. в ней перечислены названия столбцов, разделенные запятыми. Далее в каждой строке приведены значения этих столбцов, разделенные запятыми. В последней строке пропущены значения всех столбцов, кроме первого (brand).


Предположим, что текстовый файл с таким содержимым хранится в папке `data` под именем `mtcars.csv`. Для чтения такой таблицы используется метод [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). 

In [7]:
import os
path = os.path.join("data", "mtcars.csv")

df = pd.read_csv(path)
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   brand   32 non-null     object 
 1   mpg     31 non-null     float64
 2   cyl     31 non-null     float64
 3   disp    31 non-null     float64
 4   hp      31 non-null     float64
 5   drat    31 non-null     float64
 6   wt      31 non-null     float64
 7   qsec    31 non-null     float64
 8   vs      31 non-null     float64
 9   am      31 non-null     float64
 10  gear    31 non-null     float64
 11  carb    31 non-null     float64
dtypes: float64(11), object(1)
memory usage: 3.1+ KB
None


,brand,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0
1,Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0
2,Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0
3,Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0
4,Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0


В самом простом варианте использования функции `read_csv` 
- имена столбцов распознаются из первой строки файла (параметром `header` можно повлиять на это);
- в качестве индекса генерируется `RangeIndex` (параметром `index_col` можно выбрать столбец индекса);
- в качестве разделителя ожидается символ запятой (`,`) (параметром `sep` можно на это повлиять);
- пропущенные значения заполняются `np.nan`;
- столбцы с датами не распознаются.

```{note}
Метод [DataFrame.head](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) возвращает первые `n` строк таблицы. По умолчанию `n` равно 5, но можно указать явно. Похожий по смыслу метод [DataFrame.tail](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html) возвращает последние `n` строк.

Метод [DataFrame.info](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) печатает информацию о таблице. В частности, из вывода этой функции можно понять количество строк и столбцов, тип индекса таблицы, имя каждого столбца, тип данных и количество непропущенных значений в них.
```

Считаем эту таблицу ещё раз, указав в этот раз в качестве индекса столбец `brand`.

In [8]:
df = pd.read_csv(path, index_col="brand", sep=",")
df

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
brand,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0





Аналогично можно считывать данные из таблиц `excel` методом [read_excel](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html). 

Методами [to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv) и [to_excel](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html#pandas.DataFrame.to_excel) можно сохранить `DataFrame` в таблицу удобном формате (для сохранения в `excel` необходимо поставить библиотеку [openpyxl](https://openpyxl.readthedocs.io/en/stable/) или её аналоги).

## Индексация DataFrame

Доступ к строкам `DataFrame` осуществляется в точности, как осуществляется доступ к элементам `Series`, т.е. используются методы `loc` и `iloc`, которые сразу возвращают строку в виде объекта `pd.Series`, у которого в качестве индекса выступают имена столбцов.

In [9]:
df.loc["Volvo 142E"]

mpg    NaN
cyl    NaN
disp   NaN
hp     NaN
drat   NaN
wt     NaN
qsec   NaN
vs     NaN
am     NaN
gear   NaN
carb   NaN
Name: Volvo 142E, dtype: float64

Оператор `[]` датафрейма предназначен для выбора столбцов. Передавать в него можно имя нужного столбца или список таких имен. Результат в первом случае --- столбец в виде `pd.Series`, а во втором случае --- таблица `pd.DataFrame` из указанных столбцов.

In [10]:
df["mpg"].head()

brand
Mazda RX4            21.0
Mazda RX4 Wag        21.0
Datsun 710           22.8
Hornet 4 Drive       21.4
Hornet Sportabout    18.7
Name: mpg, dtype: float64

In [11]:
df[["cyl", "wt"]].tail()

,cyl,wt
brand,,
Lotus Europa,4.0,1.513
Ford Pantera L,8.0,3.170
Ferrari Dino,6.0,2.770
Maserati Bora,8.0,3.570
Volvo 142E,NaN,NaN


Если в названии столбца нет пробелов, то можно обращаться к столбцу, как к атрибуту объекта. 

In [12]:
df.carb.head(3)

brand
Mazda RX4        4.0
Mazda RX4 Wag    4.0
Datsun 710       1.0
Name: carb, dtype: float64

Методы [DataFrame.at](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) и [DataFrame.iat](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iat.html#pandas.DataFrame.iat) позволяют получить одно значение на пересечении строки и столбца.

In [13]:
df.iat[0, 0]

21.0

## Добавление столбцов 

Добавление и изменение столбцов в таблицу похоже на добавление элементов в словарь. При этом, если добавляется столбец с индексом (`pd.Series`), то данные автоматически выравниваются. 

Пусть поступили данные о стоимости автомобилей определенной марки.

In [14]:
price = pd.Series({
    "Mazda RX4": 100_000, 
    "Lotus Europa": 120_000
    })
df["price"] = price
df

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price
brand,,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0,100000.0
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0,NaN
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0,NaN
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0,NaN
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0,NaN
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0,NaN
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0,NaN
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0,NaN
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0,NaN


## Пропущенные значения

Большинство методов `pandas` разработаны таким образом, чтобы обрабатывать пропущенные значения. 

In [15]:
df.mean()

mpg          20.048387
cyl           6.258065
disp        234.261290
hp          147.903226
drat          3.580000
wt            3.231355
qsec         17.824516
vs            0.419355
am            0.387097
gear          3.677419
carb          2.838710
price    110000.000000
dtype: float64

Тем не менее иногда необходимо избавиться от строк или столбцов с пропущенными данными. Для этого удобно использовать метод [DataFrame.dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html), который возвращает новую таблицу, состоящую из строк исходной таблицы, не содержащих `np.nan`.

In [16]:
df.dropna()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price
brand,,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0,100000.0
Lotus Europa,30.4,4.0,95.1,113.0,3.77,1.513,16.90,1.0,1.0,5.0,2.0,120000.0


Параметром `axis` можно выбрать, отбрасывать ли строки или столбцы. Параметр `how` может быть равен или `all` или `any` (по умолчанию `any`) и в зависимости от этого отбрасываются те строки таблицы (столбцы), в которых пропущены все или хотя бы одно значения. 

Вместо того чтобы отбросить пропущенные значения, методом [df.fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) можно их заполнить. В самом простом варианте он заполняет `na` во всех столбцах одним и тем же значением (по умолчанию не на месте).

In [17]:
df.fillna(0)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price
brand,,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0,100000.0
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0,0.0
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0,0.0
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0,0.0
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0,0.0
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0,0.0
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0,0.0
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0,0.0
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0,0.0


Можно настроить более гибкое заполнение. Например, передавая словарь, можно выбрать свой заполнитель для каждого столбца. А параметром `method` можно заполнять пропущенные значения ближайшим непропущенным в строчках выше или ниже.

Методом [DataFrame.interpolate](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html) можно заполнить пропущенные значения интерполируя по непропущенным.

In [18]:
df.interpolate(method="linear")

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price
brand,,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0,100000.000000
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0,100740.740741
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0,101481.481481
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0,102222.222222
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0,102962.962963
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0,103703.703704
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0,104444.444444
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0,105185.185185
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0,105925.925926


## apply, map, applymap

Применить некоторую функцию к всем строкам/столбцам таблицы можно методом [DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html). Передавать этому методу необходимо такую функцию, рассчитывая, что ей на вход будет подаваться объект `pd.series`, который в зависимости от значения параметра `axis` будет или строкой или столбцом таблицы.

In [19]:
def f(x):
    return x.max() - x.min() 

df.apply(f, axis="rows").head()

mpg      23.50
cyl       4.00
disp    400.90
hp      283.00
drat      2.17
dtype: float64

Метод [Series.map](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) позволяет применить функцию к каждому значению столбца. Чтобы продемонстрировать принцип работы этого метода, извлечем из столбца `brand` марку и модель автомобиля в разные столбцы.

Для этого определим функции, которые в качестве аргумента принимают строку из столбца `brand` и возвращают марку или модель автомобиля из неё. В данной таблице, марка автомобиля всегда встречается до первого пробела, а модель автомобиля --- все после первого пробела.

In [20]:
def get_brand(x):
    return x.split(" ")[0]

def get_model(x):
    b, *m = x.split(" ") # все после первого пробела попадает в переменную m
    return " ".join(m)

x = "Mazda RX4 Wag"
print(f"{x} -> {get_brand(x)} and {get_model(x)}")

Mazda RX4 Wag -> Mazda and RX4 Wag


Определив эти функции применим их к столбцу `brand`. Т.к. `brand` --- индекс, то создадим из него `pd.Series`, применим определенные функции методом `map` и запишем результаты в таблицу в качестве новых столбцов.

In [21]:
fullname = pd.Series(df.index, index=df.index)
df.index.rename("car", inplace=True)
df["brand"] = fullname.map(get_brand)
df["model"] = fullname.map(get_model)
df

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price,brand,model
car,,,,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0,100000.0,Mazda,RX4
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0,NaN,Mazda,RX4 Wag
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0,NaN,Datsun,710
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0,NaN,Hornet,4 Drive
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0,NaN,Hornet,Sportabout
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0,NaN,Valiant,
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0,NaN,Duster,360
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0,NaN,Merc,240D
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0,NaN,Merc,230


Вместо функции, методу `map` можно передать словарь.

In [25]:
n2str = {
    4.0: "four",
    6.0: "six",
    8.0: "eight"
    }
df["cyl"] = df["cyl"].map(n2str)
df

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price,brand,model
car,,,,,,,,,,,,,,
Mazda RX4,21.0,six,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0,100000.0,Mazda,RX4
Mazda RX4 Wag,21.0,six,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0,NaN,Mazda,RX4 Wag
Datsun 710,22.8,four,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0,NaN,Datsun,710
Hornet 4 Drive,21.4,six,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0,NaN,Hornet,4 Drive
Hornet Sportabout,18.7,eight,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0,NaN,Hornet,Sportabout
Valiant,18.1,six,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0,NaN,Valiant,
Duster 360,14.3,eight,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0,NaN,Duster,360
Merc 240D,24.4,four,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0,NaN,Merc,240D
Merc 230,22.8,four,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0,NaN,Merc,230


```{warning}
Использовать числа с плавающей точкой в качестве ключей словаря считается плохой практикой из-за особенностей их представления в компьютере: даже минимальное отклонение от точного значения ключа в словаре может привести (и почти наверняка приведет) к другому значению хэш-функции и получить искомое значение по "испорченному ключу не выйдет".
```

Метод [DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html#pandas.DataFrame.applymap) применяет функцию к кажому значению в таблице.

## Группировка по значениям столбца

Часто необходимо сгруппировать строки таблицы по какому-то принципу и с каждой из групп проделать какие-то операции. За группировку в `pandas` отвечает метод [DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html). В качестве аргумента её можно передать имя столбца, по значениям которого необходима группировка (можно также передать несколько столбцов в списке).

In [113]:
grouped = df.groupby("brand")
grouped.get_group("Mazda")

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,price,brand,model
car,,,,,,,,,,,,,,
Mazda RX4,21.0,6.0,160.0,110.0,3.9,2.620,16.46,0.0,1.0,4.0,4.0,100000.0,Mazda,RX4
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.9,2.875,17.02,0.0,1.0,4.0,4.0,NaN,Mazda,RX4 Wag


Каждую группу можно рассматривать как мини-таблицу. Часто таблицу группируют по значениям одного столбца, чтобы потом посчитать какие-нибудь агрегирующие функцию к значениям других столбцов. Сделать это можно методом [DataFrameGroupBy.aggregate](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html). 

In [114]:
grouped["mpg"].aggregate(["min", "mean", "max", "count"])

,min,mean,max,count
brand,,,,
AMC,15.2,15.200000,15.2,1
Cadillac,10.4,10.400000,10.4,1
Camaro,13.3,13.300000,13.3,1
Chrysler,14.7,14.700000,14.7,1
Datsun,22.8,22.800000,22.8,1
Dodge,15.5,15.500000,15.5,1
Duster,14.3,14.300000,14.3,1
Ferrari,19.7,19.700000,19.7,1
Fiat,27.3,29.850000,32.4,2
